# Kaggle에서 MBTI 데이터 가져오기

In [1]:
conda install -c conda-forge kaggle

Solving environment: done

# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [2]:
conda install kaggle

Solving environment: done

# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install kaggle

In [5]:
!pip show kaggle

Name: kaggle
Version: 1.5.12
Summary: Kaggle API
Home-page: https://github.com/Kaggle/kaggle-api
Author: Kaggle
Author-email: support@kaggle.com
License: Apache 2.0
Location: /opt/conda/lib/python3.7/site-packages
Requires: certifi, python-dateutil, python-slugify, requests, six, tqdm, urllib3
Required-by: 


In [7]:
!kaggle config path

usage: kaggle config [-h] {view,set,unset} ...
kaggle config: error: argument command: invalid choice: 'path' (choose from 'view', 'set', 'unset')


In [9]:
!kaggle competitions list

ref                                             deadline             category            reward  teamCount  userHasEntered  
----------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                    2030-07-01 23:59:00  Getting Started     Prizes         57           False  
gan-getting-started                             2030-07-01 23:59:00  Getting Started     Prizes         83            True  
store-sales-time-series-forecasting             2030-06-30 23:59:00  Getting Started  Knowledge        598           False  
tpu-getting-started                             2030-06-03 23:59:00  Getting Started  Knowledge        117           False  
digit-recognizer                                2030-01-01 00:00:00  Getting Started  Knowledge       1346           False  
titanic                                         2030-01-01 00:00:00  Getting Started  Knowledge      14660           False  


In [10]:
!kaggle datasets list

ref                                                             title                                            size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  ----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
akshaydattatraykhare/diabetes-dataset                           Diabetes Dataset                                  9KB  2022-10-06 08:55:25          11119        350  1.0              
whenamancodes/covid-19-coronavirus-pandemic-dataset             COVID -19 Coronavirus Pandemic Dataset           11KB  2022-09-30 04:05:11           8859        283  1.0              
stetsondone/video-game-sales-by-genre                           Video Game Sales by Genre                        12KB  2022-10-31 17:56:01            795         23  1.0              
whenamancodes/credit-card-customers-prediction                  Credit Card Cust

In [11]:
!kaggle datasets download -d datasnaek/mbti-type

 86%|█████████████████████████████████▌     | 21.0M/24.4M [00:00<00:00, 108MB/s]
100%|███████████████████████████████████████| 24.4M/24.4M [00:00<00:00, 108MB/s]


In [12]:
path_to_zip_file = '/home/jupyter/mbti-type.zip'
directory_to_extract_to = '/home/jupyter/dataset'

import zipfile
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [ ]:
!pip install pandas

In [14]:
import pandas as pd

df = pd.read_csv('../dataset/mbti_1.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../dataset/mbti_1.csv'